- define a schema dictionary
- load each json into a dict

In [51]:
import json
from riskmonitor.utils.config import Config
from riskmonitor.configs.config import CFG
import os
import pandas as pd

In [28]:
def flatten_dict(dd, separator ='_', prefix =''):
    return { prefix + separator + k if prefix else k : [v] # put in a list for df creation
             for kk, vv in dd.items()
             for k, v in flatten_dict(vv, separator, kk).items()
             } if isinstance(dd, dict) else { prefix : dd }

../data/sample_articles/


In [46]:
config = Config.from_json(CFG)
data_path = config.data.path
print(data_path)

article_list = []
combined_dict = {}
for json_file_name in os.listdir(data_path):
   print(json_file_name)
   json_file_path = os.path.join(data_path, json_file_name)
   with open(json_file_path) as f:
      data = json.load(f)
   article_list.append(data)
   
for article in article_list:
   for k in article.keys():
      combined_dict[k] = tuple(d[k] for d in article_list)


../data/sample_articles/
7374406743.json
7386643546.json
7388484030.json
7388631374.json
7389650681.json


In [47]:
combined_dict

{'uri': ('7374406743', '7386643546', '7388484030', '7388631374', '7389650681'),
 'lang': ('eng', 'eng', 'eng', 'eng', 'eng'),
 'isDuplicate': (True, False, False, False, True),
 'date': ('2023-01-30',
  '2023-02-06',
  '2023-02-07',
  '2023-02-07',
  '2023-02-08'),
 'time': ('07:19:00', '12:33:00', '12:35:00', '14:03:00', '04:24:00'),
 'dateTime': ('2023-01-30T07:19:00Z',
  '2023-02-06T12:33:00Z',
  '2023-02-07T12:35:00Z',
  '2023-02-07T14:03:00Z',
  '2023-02-08T04:24:00Z'),
 'dateTimePub': ('2023-01-30T06:40:00Z',
  '2023-02-06T12:31:00Z',
  '2023-02-07T12:33:00Z',
  '2023-02-07T13:54:00Z',
  '2023-02-08T04:23:00Z'),
 'dataType': ('news', 'news', 'news', 'news', 'news'),
 'sim': (0, 0.6000000238418579, 0.4745098054409027, 0.8705882430076599, 0),
 'url': ('https://www.dailymail.co.uk/wires/ap/article-11691291/Earthquake-rocks-Chinas-northwestern-Xinjiang-region.html',
  'https://www.rt.com/news/571037-syria-rail-service-earthquake/',
  'https://www.dailymail.co.uk/news/article-11722029

In [48]:
pd.DataFrame(combined_dict)

,uri,lang,isDuplicate,date,time,dateTime,dateTimePub,dataType,sim,url,title,body,source,authors,image,eventUri,sentiment,wgt,relevance
0,7374406743,eng,True,2023-01-30,07:19:00,2023-01-30T07:19:00Z,2023-01-30T06:40:00Z,news,0.000000,https://www.dailymail.co.uk/wires/ap/article-1...,Earthquake rocks China's northwestern Xinjiang...,BEIJING (AP) - Residents and travelers sought ...,"{'uri': 'dailymail.co.uk', 'dataType': 'news',...",[],None,None,-0.239216,5,5
1,7386643546,eng,False,2023-02-06,12:33:00,2023-02-06T12:33:00Z,2023-02-06T12:31:00Z,news,0.600000,https://www.rt.com/news/571037-syria-rail-serv...,Syria stops rail service -- RT World News,Rail traffic has been suspended across Syria a...,"{'uri': 'rt.com', 'dataType': 'news', 'title':...",[],https://mf.b37mrtl.ru/files/2023.02/article/63...,eng-8381231,-0.286275,4,4
2,7388484030,eng,False,2023-02-07,12:35:00,2023-02-07T12:35:00Z,2023-02-07T12:33:00Z,news,0.474510,https://www.dailymail.co.uk/news/article-11722...,"Turkey earthquake: Will flights be cancelled, ...",The country has been devastated by three earth...,"{'uri': 'dailymail.co.uk', 'dataType': 'news',...",[],https://i.dailymail.co.uk/1s/2023/02/07/12/674...,eng-8380242,-0.474510,25,25
3,7388631374,eng,False,2023-02-07,14:03:00,2023-02-07T14:03:00Z,2023-02-07T13:54:00Z,news,0.870588,https://ca.sports.yahoo.com/news/1-shipping-co...,UPDATE 1-Shipping containers ablaze at Turkey'...,"ISKENDERUN, Turkey, Feb 7 (Reuters) - Hundreds...","{'uri': 'ca.sports.yahoo.com', 'dataType': 'ne...",[],https://s.yimg.com/cv/apiv2/social/images/yaho...,eng-8384324,-0.262745,7,7
4,7389650681,eng,True,2023-02-08,04:24:00,2023-02-08T04:24:00Z,2023-02-08T04:23:00Z,news,0.000000,https://ca.news.yahoo.com/shipping-containers-...,Fire at Turkey's Iskenderun Port extinguished ...,"ISKENDERUN, Turkey (Reuters) -A fire that engu...","{'uri': 'ca.news.yahoo.com', 'dataType': 'news...",[],https://media.zenfs.com/en/reuters.com/197d11d...,None,-0.443137,6,6


In [57]:
class DataLoader:

    @staticmethod
    def load_json(data_config):
        """Loads json files of common keys in a specified folder path and returns the following:
        a list of flattened article dictionaries, a flattened dictionary of combined common keys, and a pandas dataframe"""
        data_path = data_config.path
        article_list = []
        article_dict = {}
        for json_file_name in os.listdir(data_path):
            json_file_path = os.path.join(data_path, json_file_name)
            with open(json_file_path) as f:
                data = json.load(f)
            article_list.append(data)
        for article in article_list:
            for k in article.keys():
                article_dict[k] = tuple(d[k] for d in article_list)
        article_df = pd.DataFrame(article_dict)
        return article_list, article_dict, article_df
    
    @staticmethod
    def process_data(data_config):
        pass

In [56]:
data_loader = DataLoader()
config = Config.from_json(CFG)
data_config = config.data
_,_,article_df = DataLoader().load_json(data_config=data_config)
article_df

,uri,lang,isDuplicate,date,time,dateTime,dateTimePub,dataType,sim,url,title,body,source,authors,image,eventUri,sentiment,wgt,relevance
0,7374406743,eng,True,2023-01-30,07:19:00,2023-01-30T07:19:00Z,2023-01-30T06:40:00Z,news,0.000000,https://www.dailymail.co.uk/wires/ap/article-1...,Earthquake rocks China's northwestern Xinjiang...,BEIJING (AP) - Residents and travelers sought ...,"{'uri': 'dailymail.co.uk', 'dataType': 'news',...",[],None,None,-0.239216,5,5
1,7386643546,eng,False,2023-02-06,12:33:00,2023-02-06T12:33:00Z,2023-02-06T12:31:00Z,news,0.600000,https://www.rt.com/news/571037-syria-rail-serv...,Syria stops rail service -- RT World News,Rail traffic has been suspended across Syria a...,"{'uri': 'rt.com', 'dataType': 'news', 'title':...",[],https://mf.b37mrtl.ru/files/2023.02/article/63...,eng-8381231,-0.286275,4,4
2,7388484030,eng,False,2023-02-07,12:35:00,2023-02-07T12:35:00Z,2023-02-07T12:33:00Z,news,0.474510,https://www.dailymail.co.uk/news/article-11722...,"Turkey earthquake: Will flights be cancelled, ...",The country has been devastated by three earth...,"{'uri': 'dailymail.co.uk', 'dataType': 'news',...",[],https://i.dailymail.co.uk/1s/2023/02/07/12/674...,eng-8380242,-0.474510,25,25
3,7388631374,eng,False,2023-02-07,14:03:00,2023-02-07T14:03:00Z,2023-02-07T13:54:00Z,news,0.870588,https://ca.sports.yahoo.com/news/1-shipping-co...,UPDATE 1-Shipping containers ablaze at Turkey'...,"ISKENDERUN, Turkey, Feb 7 (Reuters) - Hundreds...","{'uri': 'ca.sports.yahoo.com', 'dataType': 'ne...",[],https://s.yimg.com/cv/apiv2/social/images/yaho...,eng-8384324,-0.262745,7,7
4,7389650681,eng,True,2023-02-08,04:24:00,2023-02-08T04:24:00Z,2023-02-08T04:23:00Z,news,0.000000,https://ca.news.yahoo.com/shipping-containers-...,Fire at Turkey's Iskenderun Port extinguished ...,"ISKENDERUN, Turkey (Reuters) -A fire that engu...","{'uri': 'ca.news.yahoo.com', 'dataType': 'news...",[],https://media.zenfs.com/en/reuters.com/197d11d...,None,-0.443137,6,6
